# Fine-tuning Classifier LLM


In [1]:
!pip install optuna
!pip install typing
!pip install evaluate
!pip install torch
!pip install transformers
pip install accelerate>=0.26.0

In [1]:
# setup - load packages
import pandas as pd
from datasets import Dataset, load_dataset
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    balanced_accuracy_score
)
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from typing import Union, Mapping, List, Dict, Any
import evaluate
from tqdm import tqdm
import zipfile
import os
import accelerate


# Set up device (is available use GPU to speed up computations)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

seed = 13

cuda


## VERSION B

In [2]:
classifier_data = pd.read_csv("../data/classifier_data_B.csv")
# converting to huggingface dataset format
data = Dataset.from_pandas(classifier_data)
# splitting into train, test and validation sets
# party data
raw_dataset = data.shuffle(seed=seed)

# 70% train, 15% test, 15% validation data
split = raw_dataset.train_test_split(test_size=0.3, seed=seed)
train_data = split["train"]
text_and_val_data = split["test"]
split = text_and_val_data.train_test_split(test_size=0.5, seed=seed)
test_data = split["train"]
val_data = split["test"]

print(f"Training samples party: {len(train_data)}")
print(f"Test samples party: {len(test_data)}")
print(f"Validation samples party: {len(val_data)}")



# data balancing??


Training samples party: 25281
Test samples party: 5418
Validation samples party: 5418


In [3]:
WINDOW_LENGTH = 512
STRIDE = 256

In [4]:
# Load Tokenizer
model_name = "bert-base-german-cased"
num_labels = 6
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    model_max_length=WINDOW_LENGTH
)


In [5]:
label_names = ['CDU/CSU', 'SPD', 'GRÜNE', 'FDP', 'AfD', 'LINKE']
label2id = {label: i for i, label in enumerate(sorted(label_names))}
id2label = {i: label for label, i in label2id.items()}

In [6]:
def sliding_window_tokenize(batch):
    texts = batch["speech_text"]
    labels = batch["label"]  # ensure this is a flat list of ints

    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=WINDOW_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
    )

    # Assign each overflow window the correct label
    tokenized["labels"] = [label2id[labels[i]] for i in tokenized["overflow_to_sample_mapping"]]

    return tokenized


In [7]:
tokenized_train_data = train_data.map(
    sliding_window_tokenize,
    batched=True,
    remove_columns=train_data.column_names
)

tokenized_val_data = val_data.map(
    sliding_window_tokenize,
    batched=True,
    remove_columns=train_data.column_names
)

Map:   0%|          | 0/25281 [00:00<?, ? examples/s]

Map:   0%|          | 0/5418 [00:00<?, ? examples/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-german-cased",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id
)

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"]

    return {
        "accuracy": acc,
        "f1": f1
    }


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Hyperparameter Tuning

In [9]:
# training arguments for hyperparameter tuning
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",  # don't save checkpoints during tuning
    logging_dir="./logs",
    disable_tqdm=True,  # speed up tuning
    report_to="none",   # optional: disable W&B or other logging
)

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 4),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
    }

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

best_run = trainer.hyperparameter_search(
    direction="maximize",        # because we want to maximize accuracy
    hp_space=hp_space,
    n_trials=12,                 # how many combinations to try
    compute_objective=lambda metrics: metrics["eval_accuracy"],
    backend="optuna"
)

print("Best hyperparameters:")
print(best_run.hyperparameters)


/tmp/ipykernel_2676/3039629573.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-08-01 16:57:57,777] A new study created in memory with name: no-name-da855184-2fc6-43f4-86d0-16055b932a64
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_

{'loss': 1.4537, 'grad_norm': 5.947567939758301, 'learning_rate': 1.8528543363470492e-05, 'epoch': 0.5995203836930456}
{'eval_loss': 1.061736822128296, 'eval_accuracy': 0.5855413230391296, 'eval_f1': 0.5856789812325323, 'eval_runtime': 80.5806, 'eval_samples_per_second': 70.724, 'eval_steps_per_second': 8.848, 'epoch': 1.0}


/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.1109, 'grad_norm': 10.283782958984375, 'learning_rate': 1.526302638013061e-05, 'epoch': 1.1990407673860912}
{'loss': 0.9223, 'grad_norm': 16.60808753967285, 'learning_rate': 1.1997509396790729e-05, 'epoch': 1.7985611510791366}
{'eval_loss': 0.9230913519859314, 'eval_accuracy': 0.6378311984558694, 'eval_f1': 0.6322667488641837, 'eval_runtime': 79.7667, 'eval_samples_per_second': 71.446, 'eval_steps_per_second': 8.939, 'epoch': 2.0}


/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

{'loss': 0.7444, 'grad_norm': 13.95425033569336, 'learning_rate': 9.355764856754166e-06, 'epoch': 2.0995800839832035}
{'loss': 0.5967, 'grad_norm': 20.677427291870117, 'learning_rate': 6.241329152108971e-06, 'epoch': 2.3995200959808036}
{'loss': 0.5904, 'grad_norm': 30.512723922729492, 'learning_rate': 3.1268934474637765e-06, 'epoch': 2.699460107978404}
{'loss': 0.5582, 'grad_norm': 7.648320198059082, 'learning_rate': 1.2457742818580782e-08, 'epoch': 2.9994001199760048}
{'eval_loss': 0.9311506152153015, 'eval_accuracy': 0.6639761361642393, 'eval_f1': 0.6666455838850364, 'eval_runtime': 37.8357, 'eval_samples_per_second': 150.625, 'eval_steps_per_second': 18.845, 'epoch': 3.0}
{'train_runtime': 1534.4917, 'train_samples_per_second': 52.12, 'train_steps_per_second': 3.259, 'train_loss': 0.9100847143133839, 'epoch': 3.0}


[I 2025-08-02 01:11:18,660] Trial 10 finished with value: 0.6639761361642393 and parameters: {'learning_rate': 3.1150585917861244e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.23100132920522845}. Best is trial 7 with value: 0.6708194420073698.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.4536, 'grad_norm': 3.9133267402648926, 'learning_rate': 2.3391802485445706e-05, 'epoch': 0.5995203836930456}


[I 2025-08-02 01:19:46,779] Trial 11 pruned. 


{'eval_loss': 1.044995665550232, 'eval_accuracy': 0.5892261800315844, 'eval_f1': 0.5860441082037408, 'eval_runtime': 37.5913, 'eval_samples_per_second': 151.604, 'eval_steps_per_second': 18.967, 'epoch': 1.0}
Best hyperparameters:
{'learning_rate': 4.058670637179588e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 4, 'weight_decay': 0.18336963954777766}


In [10]:
best_run_A_df = pd.DataFrame(best_run)
best_run_A_df.to_csv("hyperpara_B.csv",index=False)

## Training with best Tuning Parameters


In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_run.hyperparameters["learning_rate"],
    per_device_train_batch_size=best_run.hyperparameters["per_device_train_batch_size"],
    num_train_epochs=best_run.hyperparameters["num_train_epochs"],
    weight_decay=best_run.hyperparameters["weight_decay"],
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipykernel_2676/3548648206.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.409700,1.007001,0.610984,0.603266
2,0.865900,0.912571,0.639937,0.639139
3,0.584100,0.910643,0.675733,0.671737
4,0.372200,0.992627,0.675382,0.675238


/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=3336, training_loss=0.7817067025090865, metrics={'train_runtime': 455.3439, 'train_samples_per_second': 234.188, 'train_steps_per_second': 7.326, 'total_flos': 2.805811815326515e+16, 'train_loss': 0.7817067025090865, 'epoch': 4.0})

In [12]:
model.save_pretrained("classifier_final_B/")
tokenizer.save_pretrained("classifier_final_B/")

('classifier_final_B/tokenizer_config.json',
 'classifier_final_B/special_tokens_map.json',
 'classifier_final_B/vocab.txt',
 'classifier_final_B/added_tokens.json',
 'classifier_final_B/tokenizer.json')

In [13]:
# model und tokenizer müssen schon geladen sein
model.eval()

def tokenize_sliding_windows(example: Dict[str, Any]) -> Dict[str, Any]:
    encoding = tokenizer(
        example["speech_text"],
        truncation=True,
        padding="max_length",
        max_length=WINDOW_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=False,
        return_tensors="pt"
    )
    return encoding

def predict_proba_for_dataset(dataset: Dataset, label_names) -> List[Dict[str, Any]]:
    results = []

    for example in tqdm(dataset):
        tokenized = tokenize_sliding_windows(example)
        input_ids = tokenized["input_ids"].to(model.device)
        attention_mask = tokenized["attention_mask"].to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()

        avg_probs = probs.mean(axis=0)
        results.append({
            "probs": avg_probs.tolist(),
            "label": example["label"]  # falls du wahren Wert mitgeben willst
        })

    for item in results:
      probs = item["probs"]
      pred_idx = int(np.argmax(probs))
      item["prediction_label"] = label_names[pred_idx]
    return results


### Validation

In [14]:
results_val_B = predict_proba_for_dataset(val_data, sorted(label_names))

  0%|          | 0/5418 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 5418/5418 [00:35<00:00, 153.21it/s]


In [15]:
results_val_B_df = pd.DataFrame(results_val_B)

In [16]:
results_val_B_df.to_csv("classifier_final_B_validation_results.csv",index=False)

### TEST

In [17]:
results_test_B = predict_proba_for_dataset(test_data, sorted(label_names))

100%|██████████| 5418/5418 [00:35<00:00, 153.34it/s]


In [18]:
results_test_B_df = pd.DataFrame(results_test_B)

In [19]:
results_test_B_df.to_csv("classifier_final_B_test_results.csv", index=False)

In [20]:

# Name of the zip file you want to create
zip_filename = "allresultsB.zip"

# Create a zip file
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add model/tokenizer folder
    for root, dirs, files in os.walk("classifier_final_B/"):
        for file in files:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, start=os.path.dirname("classifier_final_B/"))
            zipf.write(filepath, arcname=arcname)

    # Add any CSVs you want
    for csv_file in ["hyperpara_B.csv", "classifier_final_B_validation_results.csv", "classifier_final_B_test_results.csv"]:
        if os.path.exists(csv_file):
            zipf.write(csv_file)